# I want to compare the magnitudes/fluxes of a WD in two photometric systems.

In [1]:
import pandas as pd
import scipy.interpolate as interp

sdss_DA = pd.read_csv("tables/Bergeron_DA2.txt", delim_whitespace=True, skiprows=0, header=1)
hcam_DA = pd.read_csv("/home/php18jfw/Github/lfit_python/Bergeron/Table_DA", delim_whitespace=True, skiprows=0, header=1)

In [2]:
print(sdss_DA.columns)
sdss_DA.head()

Index(['Teff', 'log_g', 'M/Mo', 'Mbol', 'BC', 'Age', 'U', 'B', 'V', 'R', 'I',
       'J', 'H', 'K', 'IR1', 'IR2', 'IR3', 'IR4', 'u_s', 'g_s', 'r_s', 'i_s',
       'z_s', 'KG5'],
      dtype='object')


,Teff,log_g,M/Mo,Mbol,BC,Age,U,B,V,R,...,IR1,IR2,IR3,IR4,u_s,g_s,r_s,i_s,z_s,KG5
0,1500.0,7.0,0.150,19.064,0.983,1.623000e+10,23.034,20.606,18.081,17.823,...,16.273,15.848,15.703,15.233,24.045,19.305,17.694,19.779,19.447,18.780
1,1750.0,7.0,0.151,18.393,0.778,1.338000e+10,22.268,20.039,17.615,17.167,...,15.681,15.323,15.210,14.843,23.262,18.813,17.082,18.552,18.333,18.198
2,2000.0,7.0,0.151,17.812,0.538,1.078000e+10,21.615,19.573,17.274,16.667,...,15.208,14.903,14.802,14.502,22.589,18.421,16.656,17.544,17.410,17.755
3,2250.0,7.0,0.151,17.297,0.310,7.879000e+09,21.036,19.165,16.988,16.260,...,14.803,14.540,14.441,14.186,21.990,18.080,16.338,16.734,16.643,17.391
4,2500.0,7.0,0.152,16.837,0.094,5.479000e+09,20.524,18.805,16.743,15.929,...,14.449,14.221,14.120,13.893,21.460,17.779,16.094,16.110,16.012,17.091


In [3]:
print(hcam_DA.columns)
hcam_DA.head()

Index(['Teff', 'log_g', 'M/Mo', 'Mbol', 'BC', 'U', 'B', 'V', 'R', 'I', 'J',
       'H', 'Ks', 'Y', 'J.1', 'H.1', 'K', 'W1', 'W2', 'W3', 'W4', 'S3.6',
       'S4.5', 'S5.8', 'S8.0', 'u_s', 'g_s', 'r_s', 'i_s', 'z_s', 'y', 'G',
       'G_BP', 'G_RP', 'FUV', 'NUV', 'Age'],
      dtype='object')


,Teff,log_g,M/Mo,Mbol,BC,U,B,V,R,I,...,r_s,i_s,z_s,y,G,G_BP,G_RP,FUV,NUV,Age
0,2500.0,7.0,0.152,16.837,0.101,20.582,18.827,16.736,15.930,15.740,...,16.088,16.190,16.087,16.256,16.393,16.972,15.689,51.792,30.247,5.479000e+09
1,2750.0,7.0,0.152,16.420,-0.089,20.106,18.487,16.509,15.651,15.194,...,15.876,15.674,15.560,15.705,16.044,16.746,15.259,50.271,29.456,3.943000e+09
2,3000.0,7.0,0.152,16.039,-0.246,19.637,18.151,16.286,15.409,14.779,...,15.680,15.286,15.131,15.223,15.743,16.525,14.910,48.677,28.639,3.472000e+09
3,3250.0,7.0,0.153,15.689,-0.372,19.152,17.810,16.061,15.194,14.479,...,15.490,15.002,14.800,14.827,15.489,16.302,14.637,46.903,27.776,3.145000e+09
4,3500.0,7.0,0.153,15.362,-0.464,18.650,17.456,15.827,14.993,14.255,...,15.294,14.784,14.553,14.531,15.266,16.068,14.419,44.810,26.865,2.927000e+09


In [4]:
def get_mags(DA, teff, logg, bands=None):
    '''Takes a bergeron table, and interpolates out the teff/logg from it'''
    # I need some helper stuff
    loggs = np.unique(DA['log_g'])
    teffs = np.unique(DA['Teff'])
    nlogg = len(loggs)
    nteff = len(teffs)
    
    if bands == None:
        bands = ['u_s', 'g_s', 'r_s', 'i_s', 'z_s']
    
    mags = {}
    for band in bands:
        z = np.array(DA[band])
        z = z.reshape((nlogg, nteff))

        # cubic bivariate spline interpolation on <z>
        func = interp.RectBivariateSpline(loggs, teffs, z, kx=3, ky=3)
        mag = func(logg, teff)[0,0]
        
        mags[band] = mag
    
    return mags

In [5]:
teff = 11000
logg = 7.0

hcam_mags = get_mags(hcam_DA, teff, logg)
sdss_mags = get_mags(sdss_DA, teff, logg)

print("{:^10s}|{:^10s}|{:^10s}|{:^10s}".format("Band", "HCAM", "SDSS", "Diff"))
for band in hcam_mags.keys():
    print("{:^10s}|{:^10.3f}|{:^10.3f}|{:^10.3f}".format(band, hcam_mags[band], sdss_mags[band], abs(hcam_mags[band]-sdss_mags[band])))

   Band   |   HCAM   |   SDSS   |   Diff   
   u_s    |  10.937  |  10.980  |  0.043   
   g_s    |  10.391  |  10.398  |  0.007   
   r_s    |  10.574  |  10.576  |  0.002   
   i_s    |  10.809  |  10.785  |  0.024   
   z_s    |  11.042  |  11.024  |  0.018   
